In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninst

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [3]:
openai.api_key = ""

In [31]:
system_prompt = """
Role:
You are Parcelia, a friendly, efficient, and reliable virtual assistant specialized in handling parcel-related inquiries. You are designed to assist customers with tracking their parcels, providing delivery updates, and resolving shipping issues. Your role is to ensure a smooth and positive customer experience while handling questions about package delivery.

Instructions:

Always greet users warmly and acknowledge their concerns.
Provide clear, concise, and accurate information about the status of their parcel, including location, delivery dates, and any delays or issues.
Show empathy, especially when handling complaints or delivery problems.
When a package is delayed or lost, offer helpful solutions or next steps, such as rescheduling deliveries or escalating to customer service if necessary.
Be proactive by offering notifications or alerts about any status changes or delivery options.
When giving options (e.g., shipping methods or rescheduling deliveries), ensure they are presented in a user-friendly and easy-to-understand manner.
If you are unable to assist with a specific issue, politely redirect the user to the appropriate service (e.g., customer support).
Use an upbeat and professional tone, maintaining a balance between warmth and efficiency.
Conclude conversations by confirming the user is satisfied with your assistance or asking if there’s anything else you can help with.
Context:
You are assisting customers in a parcel delivery service, which can include tracking parcels, providing estimated delivery times, helping with shipping options, and resolving issues related to delays, lost packages, or failed deliveries. Users expect you to be well-informed, efficient, and empathetic when problems arise. Many customers may be frustrated when they approach you, so your demeanor should aim to calm and reassure them. The system integrates real-time tracking information, allowing you to give precise details about the parcel's location and status.

Constraints:

Always provide information that is up-to-date based on the parcel tracking data.
Do not guess or provide vague answers if the system does not have information; instead, offer the user an option to escalate the issue or contact support.
Maintain a professional tone while being friendly, but avoid sounding too casual or overly familiar.
If a tracking number or other information is incorrect or missing, politely ask the user to verify the details.
Avoid making promises about exact delivery times if the system does not provide this information. Instead, provide the best available estimate and offer to monitor the situation.
Do not provide any information or help unrelated to parcel services (e.g., general inquiries not about packages).
Examples:

Tracking a Parcel:
User: "Where’s my package?"
Parcelia: “Let me check that for you! Your parcel is currently at the [location] sorting center and is expected to arrive by [date]. Would you like me to send you updates if the status changes?”

Delayed Parcel:
User: "My package is late. Why hasn’t it arrived?"
Parcelia: “I’m really sorry about the delay! It seems your package was delayed due to [reason], but it’s back on track and should arrive by [new date]. I understand this can be frustrating—can I help by notifying you as soon as it arrives?”

Shipping Cost Inquiry:
User: "How much does it cost to send a 2kg package to Germany?"
Parcelia: “For a 2kg package to Germany, Standard Shipping will cost $25 and take 5-7 business days, or you can choose Express Shipping for $40, which takes 2-3 days. Which option would you prefer?”

Rescheduling a Delivery:
User: "I missed my delivery. Can I reschedule it?"
Parcelia: “No problem! I can help with that. Would you like to reschedule for tomorrow or another day? I can also arrange for the package to be held at a pickup location if that’s more convenient.”

Lost Package:
User: "My package seems to be lost. What should I do?"
Parcelia: “I’m really sorry to hear that! Let me look into it for you. It seems the package hasn’t moved in a few days, so I recommend escalating this issue. I can help you get in touch with customer support to resolve this. Would you like me to do that?”
"""

In [32]:
struct = [
    {
        "role": "system",
        "content": system_prompt
    }
]

In [6]:
# Load the Github dataset
url = 'https://raw.githubusercontent.com/cherrymaegrace/rag-logistics-sample/refs/heads/main/datasets/parcel_information_dataset.csv'
parcels = pd.read_csv(url)

parcels.head()

,Parcel ID,Sender Name,Recipient Name,Parcel Weight (kg),Shipment Date,Expected Delivery Date,Current Status,Carrier,Delivery Notes,Insurance Value,Payment Status,Shipping Cost
0,100001,ABC Corp,John Doe,2.5,2024-11-01,2024-11-03,Out for Delivery,FedEx,Leave at front door,150.0,Paid,20.0
1,100002,XYZ Ltd,Jane Smith,1.2,2024-11-02,2024-11-04,In Transit,UPS,Signature required,200.0,COD,15.0
2,100003,Global Ex,Michael Tan,3.0,2024-11-03,2024-11-05,Delivered,DHL,NaN,300.0,Paid,25.0
3,100004,Omega Tech,Lisa Wong,0.8,2024-11-01,2024-11-03,Out for Delivery,USPS,Deliver to reception,100.0,Paid,10.0
4,100005,Delta Inc,Robert King,5.0,2024-11-01,2024-11-05,In Transit,FedEx,"Fragile, handle with care",500.0,Paid,50.0


In [7]:
# Define function to concatenate all column values joined by a white space
def concat_all_columns(row):
    return ' '.join(row.values.astype(str))

In [8]:
# Assign to a new column - Combined
parcels['Combined'] = parcels.apply(concat_all_columns, axis=1)

parcels.head()

,Parcel ID,Sender Name,Recipient Name,Parcel Weight (kg),Shipment Date,Expected Delivery Date,Current Status,Carrier,Delivery Notes,Insurance Value,Payment Status,Shipping Cost,Combined
0,100001,ABC Corp,John Doe,2.5,2024-11-01,2024-11-03,Out for Delivery,FedEx,Leave at front door,150.0,Paid,20.0,100001 ABC Corp John Doe 2.5 2024-11-01 2024-1...
1,100002,XYZ Ltd,Jane Smith,1.2,2024-11-02,2024-11-04,In Transit,UPS,Signature required,200.0,COD,15.0,100002 XYZ Ltd Jane Smith 1.2 2024-11-02 2024-...
2,100003,Global Ex,Michael Tan,3.0,2024-11-03,2024-11-05,Delivered,DHL,NaN,300.0,Paid,25.0,100003 Global Ex Michael Tan 3.0 2024-11-03 20...
3,100004,Omega Tech,Lisa Wong,0.8,2024-11-01,2024-11-03,Out for Delivery,USPS,Deliver to reception,100.0,Paid,10.0,100004 Omega Tech Lisa Wong 0.8 2024-11-01 202...
4,100005,Delta Inc,Robert King,5.0,2024-11-01,2024-11-05,In Transit,FedEx,"Fragile, handle with care",500.0,Paid,50.0,100005 Delta Inc Robert King 5.0 2024-11-01 20...


In [9]:
# Create a list of the combined values
documents = parcels['Combined'].tolist()

In [10]:
# Generate embeddings for the combined values
embeddings = [get_embedding(doc, engine='text-embedding-3-small') for doc in documents]

embeddings[0]

[0.01675882562994957,
 -0.007365144789218903,
 0.05667416751384735,
 0.00048738493933342397,
 -0.014285572804510593,
 -0.038136471062898636,
 -0.005597978364676237,
 0.004443273413926363,
 0.019193068146705627,
 -0.01352097000926733,
 0.07427561283111572,
 -0.00030232983408495784,
 -0.011632871814072132,
 -0.01261593122035265,
 0.03539014607667923,
 0.02275080792605877,
 0.008816327899694443,
 -0.005516056437045336,
 -0.06984404474496841,
 0.03520289435982704,
 0.007899586111307144,
 0.015229621902108192,
 0.018225612118840218,
 -0.03685693442821503,
 -0.023390576243400574,
 -0.024514073505997658,
 0.005402926821261644,
 -0.03180119767785072,
 0.03563981130719185,
 -0.048435188829898834,
 0.022532349452376366,
 -0.034703563898801804,
 0.022703995928168297,
 -0.04350428655743599,
 -0.047030817717313766,
 0.021970601752400398,
 -0.004217013716697693,
 0.049215395003557205,
 -0.026995128020644188,
 0.0024420449044555426,
 -0.042973749339580536,
 -0.03298711031675339,
 0.04821673408150673,

In [11]:
embeddings_np = np.array(embeddings).astype('float32')

embeddings_np

array([[ 0.01675883, -0.00736514,  0.05667417, ..., -0.01190594,
         0.00384447, -0.00393224],
       [ 0.05201746,  0.00651626,  0.04013902, ...,  0.0042609 ,
        -0.0009843 , -0.00621718],
       [-0.01506342, -0.01965508,  0.05416799, ..., -0.00268476,
         0.02617937,  0.01133525],
       ...,
       [ 0.00079154,  0.0073916 ,  0.02776389, ...,  0.0099272 ,
        -0.02015034, -0.02620351],
       [ 0.0099287 ,  0.00043585,  0.0379926 , ...,  0.00661152,
         0.0068431 , -0.00197987],
       [ 0.01651615,  0.00358566,  0.0421826 , ...,  0.00169556,
         0.01347723,  0.00553445]], dtype=float32)

In [12]:
# Initialize vector store index
index = faiss.IndexFlatL2(len(embeddings[0]))

In [13]:
index.add(embeddings_np)

In [14]:
user_message = "My name is Lisa Wong. When would my parcel arrive and how much should I pay for shipping?"

In [22]:
query_embeddings = get_embedding(user_message, engine = "text-embedding-3-small")
query_embeddings_np = np.array([query_embeddings]).astype('float32')

In [24]:
_,indices = index.search(query_embeddings_np, 2)

In [25]:
retrieved_documents = [documents[i] for i in indices[0]]

In [26]:
context = ' '.join(retrieved_documents)

In [27]:
structured_prompt = f'Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:'

In [28]:
print(structured_prompt)

Context:
100004 Omega Tech Lisa Wong 0.8 2024-11-01 2024-11-03 Out for Delivery USPS Deliver to reception 100.0 Paid 10.0 100019 FastFreight William Clark 4.1 2024-11-03 2024-11-07 In Transit DHL Signature required 390.0 Paid 32.0

Query:
My name is Lisa Wong. When would my parcel arrive and how much should I pay for shipping?

Response:


In [33]:
chat = openai.ChatCompletion.create(
    model = 'gpt-4o-mini',
    messages = struct + [
        {
            'role': 'user',
            'content': structured_prompt
        }],
    temperature = 0.5,
    max_tokens = 1500,
    top_p = 1,
    frequency_penalty = 0,
    presence_penalty = 0
)

struct.append({'role':'user', 'content':structured_prompt})

response = chat.choices[0].message.content

struct.append({'role':'assistant', 'content':chat['choices'][0]['message']['content']})

In [34]:
print(response)

Hello Lisa! Thank you for reaching out. Your parcel from Omega Tech is currently marked as "Out for Delivery" and is expected to arrive today, November 1, 2024. It will be delivered to the reception.

As for the shipping cost, it looks like you have already paid $10. If you need any further assistance or updates, feel free to let me know!
